In [ ]:
!pip install tensorflow_decision_forests

In [ ]:
# !pip install shap

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

after_flatten = "https://github.com/iMnuelll/Bangkit_Capstone/raw/main/Preprocessing/data_gform_after_flatten.csv"
concate_data = "https://github.com/iMnuelll/Bangkit_Capstone/raw/main/Data%20Dummy/concate_df.csv"
data_gform = "https://github.com/iMnuelll/Bangkit_Capstone/raw/main/Preprocessing/data_gform.csv"
data_gform_before_encode = '/content/data_gform_before_encode.csv'
data_dummy = '/content/new_dummy (2).csv'
dummy_gform = '/content/dataset_akhir_final.csv'
productive_time_chronotypes = '/content/productive_time_chronotypes.csv'

data = pd.read_csv(dummy_gform, sep=',')
# data = pd.read_csv(data_dummy, sep=',')

In [ ]:
data.columns
print(data.dtypes)

gender                  int64
age                     int64
job                     int64
task                    int64
work_days               int64
difficulty              int64
average_work_hour       int64
average_rest_float    float64
average_rest            int64
average_rest_round      int64
mood_before_work        int64
mood_after_work         int64
deadline                int64
importance              int64
sleep_average           int64
urgency                 int64
total_gangguan          int64
productive_time         int64
work_hour_label         int64
dtype: object


In [ ]:
# Define the features and target label
features = [
            # 'gender',
            'age',
            # 'job',
            'task',
            # 'difficulty',
            'average_rest',
            # 'average_rest_round',
            'mood_before_work',
            # 'mood_after_work',
            'deadline',
            'importance',
            'sleep_average',
            'urgency',
            'total_gangguan',
            # 'work_hour_label',
            'average_work_hour',
            'work_days',
            ]

target = 'productive_time'

X = data[features]
y = data[target]

# Encode categorical variables
# X = pd.get_dummies(X)
# le = LabelEncoder()
# y = le.fit_transform(y)

print(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=2,
                                                    stratify=y,
                                                    )
# Standardize the features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

     age  task  average_rest  mood_before_work  deadline  importance  \
0     40     6             4                 1         3           3   
1     30     5             3                 2         4           4   
2     31     5             3                 0         4           3   
3     39     5             3                 1         3           1   
4     31     3             4                 1         4           4   
..   ...   ...           ...               ...       ...         ...   
322   20     6             1                 0         1           2   
323   17     6             1                 0         1           4   
324   21     6             1                 1         1           2   
325   20     6             1                 1         1           2   
326   19     6             1                 1         2           1   

     sleep_average  urgency  total_gangguan  average_work_hour  work_days  
0                8        3               3                

In [ ]:
train_data = X_train.copy()
train_data[target] = y_train

test_data = X_test.copy()
test_data[target] = y_test

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, label=target)
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_data, label=target)

In [ ]:
results = []
best_accuracy = 0
best_config = {}

for max_depth in [None, 10, 20, 30]:
    for min_examples in [1, 2, 5, 10]:
        for num_trees in [50, 100, 200]:
            for compute_oob in [True, False]:
                model_gbt = tfdf.keras.RandomForestModel(
                    max_depth=max_depth,
                    min_examples=min_examples,
                    num_trees=num_trees,
                    compute_oob_variable_importances=compute_oob,
                    verbose=0
                )
                model_gbt.compile(metrics=['accuracy'])
                model_gbt.fit(x=train_ds)
                evaluation = model_gbt.evaluate(test_ds, return_dict=True)
                accuracy = evaluation['accuracy']
                results.append((accuracy, max_depth, min_examples, num_trees, compute_oob))
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_config = {
                        'max_depth': max_depth,
                        'min_examples': min_examples,
                        'num_trees': num_trees,
                        'compute_oob_variable_importances': compute_oob
                    }
                # Print the parameters and the accuracy for each iteration
                print(f"Max Depth: {max_depth}, Min Examples: {min_examples}, Num Trees: {num_trees}, Compute OOB: {compute_oob}, Accuracy: {accuracy:.4f}")

# Print the best configuration and its accuracy
print(f"Best Configuration: {best_config}")
print(f"Best Accuracy: {best_accuracy:.4f}")

1/1 [==============================] - 6s 6s/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 1, Num Trees: 50, Compute OOB: True, Accuracy: 0.8636
1/1 [==============================] - 0s 242ms/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 1, Num Trees: 50, Compute OOB: False, Accuracy: 0.8636
1/1 [==============================] - 0s 240ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 1, Num Trees: 100, Compute OOB: True, Accuracy: 0.8333
1/1 [==============================] - 0s 259ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 1, Num Trees: 100, Compute OOB: False, Accuracy: 0.8333


1/1 [==============================] - 0s 231ms/step - loss: 0.0000e+00 - accuracy: 0.8485
Max Depth: None, Min Examples: 1, Num Trees: 200, Compute OOB: True, Accuracy: 0.8485


1/1 [==============================] - 0s 491ms/step - loss: 0.0000e+00 - accuracy: 0.8485
Max Depth: None, Min Examples: 1, Num Trees: 200, Compute OOB: False, Accuracy: 0.8485
1/1 [==============================] - 0s 231ms/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 2, Num Trees: 50, Compute OOB: True, Accuracy: 0.8636
1/1 [==============================] - 0s 241ms/step - loss: 0.0000e+00 - accuracy: 0.8636
Max Depth: None, Min Examples: 2, Num Trees: 50, Compute OOB: False, Accuracy: 0.8636
1/1 [==============================] - 0s 248ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 2, Num Trees: 100, Compute OOB: True, Accuracy: 0.8333
1/1 [==============================] - 0s 231ms/step - loss: 0.0000e+00 - accuracy: 0.8333
Max Depth: None, Min Examples: 2, Num Trees: 100, Compute OOB: False, Accuracy: 0.8333
1/1 [==============================] - 0s 239ms/step - loss: 0.0000e+00 - accuracy: 0.8485
Max Depth: None, Min Ex

In [ ]:
# model_gbt = tfdf.keras.GradientBoostedTreesModel()
model_gbt = tfdf.keras.RandomForestModel(
    max_depth=10,
    min_examples=5,
    num_trees=100,
    verbose=1,
    compute_oob_variable_importances=True
    )

model_gbt.fit(x=train_ds)

Use /tmp/tmpskr2el54 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.553554. Found 261 examples.
Training model...
Model trained in 0:00:00.180071
Compiling model...
Model compiled.


In [ ]:
model_gbt.compile(metrics=['accuracy'])

evaluation  = model_gbt.evaluate(train_ds, return_dict = True)

for name, value in evaluation.items() :
  print(f"train {name}: {value:.4f}")

evaluation  = model_gbt.evaluate(test_ds, return_dict = True)

for name, value in evaluation.items() :
  print(f"test {name}: {value:.4f}")

1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.9464
train loss: 0.0000
train accuracy: 0.9464
1/1 [==============================] - 1s 1s/step - loss: 0.0000e+00 - accuracy: 0.8788
test loss: 0.0000
test accuracy: 0.8788


In [ ]:
inspector = model_gbt.make_inspector()
feature_importances = inspector.variable_importances()

In [ ]:
for importance_type, importance_list in feature_importances.items():
    print(f"\nImportance type: {importance_type}")
    for feature_importance in importance_list:
        print(f"  Feature: {feature_importance[0]}, Importance: {feature_importance[1]}")


Importance type: MEAN_DECREASE_IN_ACCURACY
  Feature: "age" (1; #1), Importance: 0.10344827175140381
  Feature: "deadline" (1; #4), Importance: 0.10344827175140381
  Feature: "sleep_average" (1; #7), Importance: 0.09195399284362793
  Feature: "average_work_hour" (1; #3), Importance: 0.05747121572494507
  Feature: "urgency" (1; #10), Importance: 0.015325665473937988
  Feature: "average_rest" (1; #2), Importance: 0.011494219303131104
  Feature: "total_gangguan" (1; #9), Importance: 0.011494219303131104
  Feature: "importance" (1; #5), Importance: 0.007662832736968994
  Feature: "task" (1; #8), Importance: 0.007662832736968994
  Feature: "mood_before_work" (1; #6), Importance: 0.0038313865661621094
  Feature: "work_days" (1; #11), Importance: -0.011494278907775879

Importance type: MEAN_DECREASE_IN_AP_3_VS_OTHERS
  Feature: "age" (1; #1), Importance: 0.3472863812892289
  Feature: "deadline" (1; #4), Importance: 0.09663446520392505
  Feature: "sleep_average" (1; #7), Importance: 0.0815373

**Penjelasan Importance Keys :**

https://medium.com/@shahcheril311/training-tree-based-models-with-tf-decision-forests-5eb98860c155

In [ ]:
print(model_gbt.summary())

Model: "random_forest_model_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (11):
	age
	average_rest
	average_work_hour
	deadline
	importance
	mood_before_work
	sleep_average
	task
	total_gangguan
	urgency
	work_days

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.     "sleep_average"  0.345398 ################
    2.               "age"  0.307542 ############
    3.          "deadline"  0.298218 ############
    4. "average_work_hour"  0.207036 ####
    5.         "work_days"  0.174109 #
    6.    "total_gangguan"  0.161615 
    7.              "task"  0.160582 
    8.           "urgency"  0.159757 
    9.      "average_rest"  0.158300 
   10.  "mood_be

In [ ]:
# Contoh data untuk prediksi
predict_data = {
    'age': [29],
    'task': [5],
    'average_rest': [30],
    'mood_before_work': [3],
    'deadline': [2],
    'importance': [3],
    'sleep_average': [7],
    'urgency': [1],
    'total_gangguan': [0],
    'average_work_hour': [8],
    'work_days': [5]
}

# konversi data ke DataFrame pandas
predict_df = pd.DataFrame(predict_data)

# konversi DataFrame ke TensorFlow Dataset
predict_ds = tfdf.keras.pd_dataframe_to_tf_dataset(predict_df)

# Melakukan prediksi
predictions = model_gbt.predict(predict_ds)

print("Predictions:", predictions)
print(predictions.dtype)

1/1 [==============================] - 0s 290ms/step
Predictions: [[0.17       0.55999976 0.21000002 0.05999999]]
float32


In [ ]:
!pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
model_gbt.save('chronotype_model')

In [ ]:
!pip install tensorflowjs  # Memastikan tensorflowjs terinstal
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model ./chronotype_model ./chronotype_model_tfjs

2024-06-06 14:39:15.787825: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[INFO 24-06-06 14:39:19.2613 UTC kernel.cc:1233] Loading model from path ./chronotype_model/assets/ with prefix 85487f62118945f3
[INFO 24-06-06 14:39:19.2930 UTC decision_forest.cc:734] Model loaded with 200 root(s), 12962 node(s), and 11 input feature(s).
[INFO 24-06-06 14:39:19.2930 UTC abstract_model.cc:1362] Engine "RandomForestGeneric" built
[INFO 24-06-06 14:39:19.2931 UTC kernel.cc:1061] Use fast generic engine
weight StatefulPartitionedCall/random_forest_model_194/StatefulPartitionedCall/RaggedConstant/Const with shape (1,) and dtype int64 was auto converted to the type int32
weight StatefulPartitionedCall/random_forest_model_194/StatefulPartitionedCall/RaggedConstant/Const_1 with shape (1,) and dtype int64 was auto converted to the type int32


In [ ]:
import shutil

# Path ke folder model TensorFlow.js
folder_path = './chronotype_model_tfjs'
# Nama file zip output
zip_path = './chronotype_model_tfjs.zip'

# Membuat zip file dari folder model
shutil.make_archive(zip_path.replace('.zip', ''), 'zip', folder_path)


'/content/chronotype_model_tfjs.zip'

In [ ]:
from google.colab import files

# Download file zip
files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>